In [1]:
import os
%pwd

'c:\\Users\\Asus\\Desktop\\Krish Naik MLOps Course\\Projects\\Data Science project\\end-to-end-data-science-project\\research'

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\Asus\\Desktop\\Krish Naik MLOps Course\\Projects\\Data Science project\\end-to-end-data-science-project'

## Entity

In [9]:
# entity

from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

: 

## Configuration Manager

In [4]:
from src.data_science_project.constant import *
from src.data_science_project.utils.common import read_yaml, create_directories

In [5]:

class ConfigurationManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH
                 ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        create_directories([self.config.data_ingestion.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir= self.config.data_ingestion.root_dir,
            source_url=self.config.data_ingestion.source_url,
            local_data_file=self.config.data_ingestion.local_data_file,
            unzip_dir=self.config.data_ingestion.unzip_dir
        )
        
        return data_ingestion_config
        

## Component

In [6]:
# Components- Data Ingestion

from urllib import request
import zipfile
from src.data_science_project import logger


class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with info: \n{headers}")
        else:
            logger.info(f"File already exists")
            
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as f:
            f.extractall(unzip_path)

## Pipeline

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-02-03 12:04:43,017: INFO: common: Created directory at artifacts]
[2025-02-03 12:04:43,024: INFO: common: Created directory at artifacts/data_ingestion]
[2025-02-03 12:04:44,251: INFO: 1826563597: artifacts/data_ingestion/data.zip downloaded with info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: B52A:117D:1362763:144F528:67A05C7C
Accept-Ranges: bytes
Date: Mon, 03 Feb 2025 06:04:44 GMT
Via: 1.1 varnish
X-Served-By: cache-sin-wsss1830038-SIN
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1738562685.522425,VS0,VE325
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fast